# 2024-06-15
So yea initially, I started just with a pipeline,  like

```python
from transformers import pipeline, BertTokenizer, BertModel
import torch

# Load Hugging Face pipeline for zero-shot classification
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
```

then I was reading through Jake's article, [here](https://jaketae.github.io/study/zero-shot-classification/), realizing ok the pipeline is an abstraction and you can access the raw NLI output entailment logits, `entailement, neutral, contradiction`. 

So cool, let me try this 

In [12]:
import torch

In [1]:

from transformers import BartForSequenceClassification, BartTokenizer

model_name = "facebook/bart-large-mnli"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForSequenceClassification.from_pretrained(model_name)



In [2]:
premise = "I am looking for a physical therapist who specializes in sports injuries, close by"

hypothesis = "this text is about something that is relatively near"

tokens = tokenizer(premise, hypothesis, return_tensors="pt")
outputs = model(**tokens)
outputs.keys()



odict_keys(['logits', 'past_key_values', 'encoder_last_hidden_state'])

In [3]:
logits = outputs.logits
logits.shape

torch.Size([1, 3])

In [6]:
logits

tensor([[-2.5674,  0.0570,  2.5532]], grad_fn=<AddmmBackward0>)

In [8]:
(contradiction, neutral, entailment) = float(logits[0][0]), f(logits[0][1]), logits[0][2]
(contradiction, neutral, entailment)

(tensor(-2.5674, grad_fn=<SelectBackward0>),
 tensor(0.0570, grad_fn=<SelectBackward0>),
 tensor(2.5532, grad_fn=<SelectBackward0>))

In [17]:
(torch.round(logits[0][0], decimals=3))

'tensor(-2.5670, grad_fn=<RoundBackward1>)'

In [22]:
f"{torch.round(logits, decimals=2)}" # .tolist()

'tensor([[-2.5700,  0.0600,  2.5500]], grad_fn=<RoundBackward1>)'

In [24]:
probs = logits.softmax(dim=1) ; probs

tensor([[0.0055, 0.0757, 0.9188]], grad_fn=<SoftmaxBackward0>)

In [26]:
print(f"{logits[0][0]:.3f}")
print(f"{probs[0][0]:.3f}")

-2.567
0.005


Thinking about negations hmm, maybe last time I was trying to add a negation into the "class" , aka, the premise. That was not working. Maybe this time, let's just stick to letting the neutrality, entailment, contradiction, to speak instead.

And an idea, if this doesn't work then yea maybe we'll need something rule based. or also compare to what a sophisticated high order model can say about the statements.

In [31]:
from itertools import product
hypotheses = {
    "relatively_near": "this text is about something that is relatively near",
    "specific_address": "this text refers to a specific address",
    "specific_location": "this text refers to a specific location",
    "relative_location": "this text refers to a relative location"
             }
premises = [
    "I am looking for a physical therapist who specializes in sports injuries, close by",
    "I am looking for a physical therapist who specializes in sports injuries",
    "I am looking for a physical therapist who specializes in sports injuries, nearby",
    "I am looking for a physical therapist who specializes in sports injuries, near me",
    "I am looking for a physical therapist who specializes in sports injuries, in Manhattan",
    "I am looking for a physical therapist who specializes in sports injuries, who is in NY in Brooklyn",
    "I am looking for a physical therapist who specializes in sports injuries, in the zip code 10010",
    "I am looking for a physical therapist who specializes in sports injuries, close to me",
    "I am looking for a physical therapist who specializes in sports injuries, close to Columbus Circle",
    "I am looking for a physical therapist who specializes in sports injuries, around Union Square",
]
terms = ["contradiction", "neutral", "entailment"]
for (premise, (hypothesis_brief, hypothesis)) in product(premises, hypotheses.items()):
    
    tokens = tokenizer(premise, hypothesis, return_tensors="pt")
    outputs = model(**tokens)
    logits = outputs.logits
    # (contradiction, neutral, entailment) = float(logits[0][0]), (logits[0][1]), logits[0][2]
    probs = logits.softmax(dim=1)
    # print(f"{logits[0][0]:.3f}")
    print("")
    print(premise)
    print(hypothesis)
    print([f"{x}: {probs[0][i]:.2f}" for i, x in enumerate(terms)])


I am looking for a physical therapist who specializes in sports injuries, close by
this text is about something that is relatively near
['contradiction: 0.01', 'neutral: 0.08', 'entailment: 0.92']

I am looking for a physical therapist who specializes in sports injuries, close by
this text refers to a specific address
['contradiction: 0.02', 'neutral: 0.59', 'entailment: 0.39']

I am looking for a physical therapist who specializes in sports injuries, close by
this text refers to a specific location
['contradiction: 0.01', 'neutral: 0.17', 'entailment: 0.82']

I am looking for a physical therapist who specializes in sports injuries, close by
this text refers to a relative location
['contradiction: 0.00', 'neutral: 0.12', 'entailment: 0.88']

I am looking for a physical therapist who specializes in sports injuries
this text is about something that is relatively near
['contradiction: 0.24', 'neutral: 0.55', 'entailment: 0.21']

I am looking for a physical therapist who specializes in sp

### Hmm not separating "relative" vs "specific"
I'm seeing both "specific" and "relative" getting flagged for actual locations (e.g. Union Square) 

### although better news, for the statement, without any location language, location hypotheses are more neutral 
So, not consistently, though the strenght is low on entailment here, compared with statements that have locations (relative or specific)
```
I am looking for a physical therapist who specializes in sports injuries
this text is about something that is relatively near
['contradiction: 0.24', 'neutral: 0.55', 'entailment: 0.21']

I am looking for a physical therapist who specializes in sports injuries
this text refers to a specific address
['contradiction: 0.03', 'neutral: 0.65', 'entailment: 0.32']

I am looking for a physical therapist who specializes in sports injuries
this text refers to a specific location
['contradiction: 0.05', 'neutral: 0.39', 'entailment: 0.56']

I am looking for a physical therapist who specializes in sports injuries
this text refers to a relative location
['contradiction: 0.06', 'neutral: 0.39', 'entailment: 0.55']
```

# Oh cool idea , to use a cased model fine tuned for NER ! 
lets try , per recommendation from chatgpt 

This is particularly a good idea, because these models have continuation labels, yea forgot about that, for multi-token entities.

Ah right and per [hf](https://huggingface.co/docs/transformers/en/task_summary#token-classification) , yea there are other than NER token classification, also POS part of speech token classification, which can help suss out the meat , possibly. 


In [32]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

query = "I am looking for a physical therapist who specializes in sports injuries, close by"
ner_results = nlp(query)

location_entities = [entity for entity in ner_results if entity['entity'] in ['B-LOC', 'I-LOC']]
locations = " ".join([query[entity['start']:entity['end']] for entity in location_entities])
print("Extracted Locations:", locations)


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Extracted Locations: 


In [37]:
nlp = pipeline("ner")

query = "I am looking for a physical therapist who specializes in sports injuries, close by"
ner_results = nlp(query)


No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [35]:
ner_results

[]

In [38]:
from transformers import pipeline

classifier = pipeline(task="ner")
preds = classifier("Hugging Face is a French company based in New York City.")
preds = [
    {
        "entity": pred["entity"],
        "score": round(pred["score"], 4),
        "index": pred["index"],
        "word": pred["word"],
        "start": pred["start"],
        "end": pred["end"],
    }
    for pred in preds
]
print(*preds, sep="\n")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'entity': 'I-ORG', 'score': 0.9968, 'index': 1, 'word': 'Hu', 'start': 0, 'end': 2}
{'entity': 'I-ORG', 'score': 0.9293, 'index': 2, 'word': '##gging', 'start': 2, 'end': 7}
{'entity': 'I-ORG', 'score': 0.9763, 'index': 3, 'word': 'Face', 'start': 8, 'end': 12}
{'entity': 'I-MISC', 'score': 0.9983, 'index': 6, 'word': 'French', 'start': 18, 'end': 24}
{'entity': 'I-LOC', 'score': 0.999, 'index': 10, 'word': 'New', 'start': 42, 'end': 45}
{'entity': 'I-LOC', 'score': 0.9987, 'index': 11, 'word': 'York', 'start': 46, 'end': 50}
{'entity': 'I-LOC', 'score': 0.9992, 'index': 12, 'word': 'City', 'start': 51, 'end': 55}


In [39]:
from transformers import pipeline

classifier = pipeline(task="ner")
query = "I am looking for a physical therapist who specializes in sports injuries, close by"
preds = classifier(query)
preds = [
    {
        "entity": pred["entity"],
        "score": round(pred["score"], 4),
        "index": pred["index"],
        "word": pred["word"],
        "start": pred["start"],
        "end": pred["end"],
    }
    for pred in preds
]
print(*preds, sep="\n")

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [40]:
preds

[]

Ahh ok this is probably blank, because no location entities here?? lets try another, 

In [41]:
from transformers import pipeline

classifier = pipeline(task="ner")
query = "I am looking for a physical therapist who specializes in sports injuries, around Union Square, or perhaps near Columbus Circle, or say near the south side of Central Park, or maybe just somewhere anywhere near me "

preds = classifier(query)
preds = [
    {
        "entity": pred["entity"],
        "score": round(pred["score"], 4),
        "index": pred["index"],
        "word": pred["word"],
        "start": pred["start"],
        "end": pred["end"],
    }
    for pred in preds
]
print(*preds, sep="\n")




No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'entity': 'I-LOC', 'score': 0.9918, 'index': 15, 'word': 'Union', 'start': 81, 'end': 86}
{'entity': 'I-LOC', 'score': 0.9982, 'index': 16, 'word': 'Square', 'start': 87, 'end': 93}
{'entity': 'I-LOC', 'score': 0.9884, 'index': 21, 'word': 'Columbus', 'start': 111, 'end': 119}
{'entity': 'I-LOC', 'score': 0.998, 'index': 22, 'word': 'Circle', 'start': 120, 'end': 126}
{'entity': 'I-LOC', 'score': 0.9934, 'index': 31, 'word': 'Central', 'start': 158, 'end': 165}
{'entity': 'I-LOC', 'score': 0.9975, 'index': 32, 'word': 'Park', 'start': 166, 'end': 170}


Ok that's cool because this seems to be pretty good at picking out specific geographic locations and ignore relative ones. 

How about street addresses ?

In [42]:
from transformers import pipeline

classifier = pipeline(task="ner")
query = "I'm on the corner of 14th stret and Broadway and I am trying to get to 59th street and Central Park West ok how can I travel?"

preds = classifier(query)
preds = [
    {
        "entity": pred["entity"],
        "score": round(pred["score"], 4),
        "index": pred["index"],
        "word": pred["word"],
        "start": pred["start"],
        "end": pred["end"],
    }
    for pred in preds
]
print(*preds, sep="\n")




No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'entity': 'I-LOC', 'score': 0.8308, 'index': 8, 'word': '14th', 'start': 21, 'end': 25}
{'entity': 'I-LOC', 'score': 0.9737, 'index': 13, 'word': 'Broadway', 'start': 36, 'end': 44}
{'entity': 'I-LOC', 'score': 0.7358, 'index': 21, 'word': '59', 'start': 71, 'end': 73}
{'entity': 'I-LOC', 'score': 0.749, 'index': 22, 'word': '##th', 'start': 73, 'end': 75}
{'entity': 'I-LOC', 'score': 0.9923, 'index': 25, 'word': 'Central', 'start': 87, 'end': 94}
{'entity': 'I-LOC', 'score': 0.9966, 'index': 26, 'word': 'Park', 'start': 95, 'end': 99}
{'entity': 'I-LOC', 'score': 0.9964, 'index': 27, 'word': 'West', 'start': 100, 'end': 104}
